# scRFE source V4 - kidney split by age with first 100

In [1]:
# cleaning up code after meeting with Sevahn

In [2]:
# all code broken up into separate arguments for scRFE

In [1]:
# Imports 
import numpy as np
import pandas as pd
import scanpy as sc
from anndata import read_h5ad
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV

In [15]:
# read in data 
adata = read_h5ad('/Users/madelinepark/Downloads/Kidney_facs.h5ad')
tiss = adata
# tiss.obs

In [ ]:
Subset the anndata for at most 100 of each label type (e.g. of your cell type for that iteration, then rest) -- that should make it run much faster. here's how to subset a matrix:
https://stackoverflow.com/questions/10625096/extracting-first-n-columns-of-a-numpy-matrix

general procedure
subset anndata for the label of interest and rest
get the shape e.g. anndataSubset.shape # (rows = cells, cols = genes)
if dimension of X for the # rows (e.g. anndataSubset.shape[0] < 100), use them all, else get first 100 (adata.X[0:100:,:]) -- look up how to subset matrix
then make your labels accordingly too.

In [ ]:
We only want the first 100 rows for now for each age so

In [27]:
type(adata.X[0:,:100:])
new_matrix = adata.X[0:,:100:]

In [3]:
# tiss.obs['cell_ontology_class']

In [34]:
# split data for training
def loc_split_function(tiss, feature='age'):
    tiss.obs['feature_type_of_interest'] = 'rest'
    for c in list(set(tiss.obs[feature])): 
        feature_of_interest = c
        tiss.obs.loc[tiss.obs[tiss.obs[feature] == feature_of_interest].index,'feature_type_of_interest'] = feature_of_interest
        feat_labels = tiss.var_names 
        X = tiss.X[0:,:10:] #seeing if we can take first 10 rows but idk if it does this for both
        y = tiss.obs['feature_type_of_interest']
    return X, y, feature, feat_labels #this is returning only the last thing in the loop 

In [35]:
# create random forest and selector, then train
def train_function(X, y, test_size, random_state):
    print('training...')
    loc_split = loc_split_function(tiss=tiss, feature='age')
    X = loc_split[0]
    y = loc_split[1]
    feat_labels = loc_split[3] #this should not be hardcoded #genes be argument
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=0) 
    clf = RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=-1, oob_score=True)
    selector = RFECV(clf, step=0.2, cv=3, n_jobs=4)
    clf.fit(X_train, y_train)
    selector.fit(X_train, y_train)
    feature_selected = feat_labels[selector.support_]
    return selector, clf, feat_labels, feature_selected, selector.estimator_.feature_importances_, X_train, X_test, y_train, y_test

In [36]:
cd /Users/madelinepark/src3/scRFE/scRFE-results

/Users/madelinepark/src3/scRFE/scRFE-results


In [37]:
# result write
def result_write (c, feature_selected, tiss, feature='age',test_size=0.05, random_state=0): 
    results_df = pd.DataFrame()
    print('result writing...')
    loc_split = loc_split_function(tiss=tiss, feature= feature)
    X = loc_split[0]
    y = loc_split[1]
    train = train_function(X, y, test_size, random_state)
    feat_labels = train[0] 
    feature_selected = train[3]
    gini_scores = train[4]
    column_headings = []
    column_headings.append(c)
    column_headings.append(c + '_gini')
    resaux = pd.DataFrame(columns=column_headings)
    resaux[c] = feature_selected 
    resaux[c + '_gini'] = (gini_scores) 
    results_df = pd.concat([results_df,resaux],axis=1)
    tiss.obs['feature_type_of_interest'] = 'rest'
    file_name = 'kidney facs' + feature + c + ".csv"
    results_df.sort_values(by = 'gini_index', ascending = True, axis = 0)
    print(results_df)
    results_df.to_csv(file_name) #trying to save results as a csv
    return results_df

In [38]:
# gini_3_m = result_write (c='3m', feature_selected, tiss, feature='age',test_size=0.05, random_state=0)
# gini_24_m = result_write (c='24m', feature_selected, tiss, feature='age',test_size=0.05, random_state=0)

In [39]:
# combined functions
def scRFE (tiss=tiss, X=tiss.X, feature='age', n_estimators=1000, random_state=0, n_jobs=-1, oob_score=True, test_size = 0.05, step=0.2, cv=5) :
    all_trees = []
    for c in list(set(tiss.obs[feature])): 
        print(c)
        loc_split = loc_split_function(tiss=tiss, feature= feature)
        X = loc_split[0] #change age to feature
        y = loc_split[1]
        feature = loc_split[2]
        feat_labels = loc_split[3]
        train = train_function(X, y, test_size, random_state)
        feature_selected = train[1]
        X_train = train[2]
        X_test = train[3]
        y_train = train[4]
        y_test = train[5]        
        all_trees += [result_write(c, feature_selected, tiss, feature=feature,test_size=0.05, random_state=0)]
        #returns data frames from SCRFE in order
        tiss.obs['age_type_of_interest'] = 'rest'
    return all_trees
#         results_df.to_csv('scRFE-results.csv') #trying to save results as a csv - this didnt work

In [40]:
# scRFE(tiss=tiss, feature='age', n_estimators=10, random_state=0, n_jobs=-1, oob_score=True, test_size = 0.05, step=0.2, cv=5)

In [41]:
makeAllTrees = scRFE(tiss=tiss, feature='age', n_estimators=10, random_state=0, n_jobs=-1, oob_score=True, test_size = 0.05, step=0.2, cv=5)
# age3 = makeAllTrees[1]
# age24 = makeAllTrees[0]

3m
training...


IndexError: boolean index did not match indexed array along dimension 0; dimension is 22899 but corresponding boolean dimension is 10

In [ ]:
# run function
# scRFE(tiss=tiss, feature='age', n_estimators=1000, random_state=0, n_jobs=-1, oob_score=True, test_size = 0.05, step=0.2, cv=5)

In [51]:
# how do i get it to append after so the last dataframe doesnt completely overwrite the previous one

'/Users/madelinepark/src3/scRFE/scRFE-results'

In [ ]:
# # combined functions
# def scRFE_save (tiss=tiss, X=tiss.X, feature='age', n_estimators=1000, random_state=0, n_jobs=-1, oob_score=True, test_size = 0.05, step=0.2, cv=5) :
#     for c in list(set(tiss.obs[feature])): 
#         print(c)
#         X = loc_split(tiss=tiss, feature= feature)[0] #change age to feature
#         y = loc_split(tiss=tiss, feature= feature)[1]
#         feature = loc_split(tiss=tiss, feature= feature)[2]
#         feat_labels = loc_split(tiss=tiss, feature= feature)[3]
#         feature_selected = train(X, y, test_size, random_state)[1]
#         X_train = train(X, y, test_size, random_state)[2]
#         X_test = train(X, y, test_size, random_state)[3]
#         y_train = train(X, y, test_size, random_state)[4]
#         y_test = train(X, y, test_size, random_state)[5]
#         result_write(c, feature_selected, tiss, feature=feature,test_size=0.05, random_state=0)
#         tiss.obs['age_type_of_interest'] = 'rest'
# #         not sorted